In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nn.nn import NeuralNetwork
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [ ]:
example_NN = NeuralNetwork(nn_arch = [{'input_dim': 64, 'output_dim': 16, 'activation': 'relu'},
                                      {'input_dim': 16, 'output_dim': 64, 'activation': 'relu'}],
                            lr = 0.0005, seed = 3, batch_size = 100, epochs = 500, loss_function='mse')


#params are arrays of W1-Wx where x is feature number
#and also an array for b1 

In [ ]:

per_epoch_loss_train, per_epoch_loss_val=example_NN.fit(X_train, y_train, X_val, y_val)

In [ ]:
#taken from hw7

fig, axs = plt.subplots(2, figsize=(8, 8))
axs[0].plot(np.arange(len(per_epoch_loss_train)), per_epoch_loss_train)
axs[0].set_title('Training Loss History')
axs[1].plot(np.arange(len(per_epoch_loss_val)), per_epoch_loss_val)
axs[1].set_title('Validation Loss History')
plt.xlabel('Steps')
fig.tight_layout()
plt.show()